In [2]:
!pip install --upgrade numpy
!pip install --upgrade numexpr

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [17]:
import os
import json
import traceback
import urllib.parse
import boto3
from datetime import datetime
import time
from smart_search_qa import SmartSearchQA
from prompt import *


# EMBEDDING_ENDPOINT_NAME = "huggingface-inference-eb"
# EMBEDDING_ENDPOINT_NAME = "huggingface-inference-eb-bge-zh"
EMBEDDING_ENDPOINT_NAME = "huggingface-inference-eb-bge-en"
LLM_ENDPOINT_NAME = "pytorch-inference-llm-v1"
# index =  "smart_search_qa_test"
# index = "smart_search_qa_test_0826_cn"
index = "smart_search_qa_test_0826_en_3"
search_engine = 'opensearch'
# language = "chinese"
language = "english"

# EMBEDDING_ENDPOINT_NAME = "huggingface-inference-text2vec-base-chinese-v1"
# LLM_ENDPOINT_NAME = "pytorch-inference-chatglm2-g5-2x"
# language = "chinese"
# index =  "smart_search_qa_demo_0620_cn"
# search_engine = 'opensearch'


sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-host-url')['SecretString']
data= json.loads(master_user)
es_host_name = data.get('host')
host = es_host_name+'/' if es_host_name[-1] != '/' else es_host_name# cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com/
host = host[8:-1]
region = boto3.Session().region_name # e.g. cn-north-1
print('host:',host)
print('region:',region)

sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)
username = data.get('username')
password = data.get('password')
port = 443

# EMBEDDING_ENDPOINT_NAME = "pytorch-inference-all-minilm-l6-v2"
# LLM_ENDPOINT_NAME = "pytorch-inference-vicuna-p3-2x"
# language = "english"
# host =  "bce3f35d-4471-432a-916a-d6ec42eb15a2"
# search_engine = 'kendra'
# index=''
# username=''
# password=''
# port=''
# region='us-west-2'




if language == "chinese":
    prompt_template = CHINESE_PROMPT_TEMPLATE
    condense_question_prompt = CN_CONDENSE_QUESTION_PROMPT
elif language == "english":
    prompt_template = ENGLISH_PROMPT_TEMPLATE
    condense_question_prompt = EN_CONDENSE_QUESTION_PROMPT

# model_type='normal'
model_type='bedrock'
api_url = "https://bx2kc13ys3.execute-api.us-east-1.amazonaws.com/prod/bedrock?"
    

search_qa = SmartSearchQA()
search_qa.init_cfg(index,
                 username,
                 password,
                 host,
                 port,
                 EMBEDDING_ENDPOINT_NAME,
                 region,
                 LLM_ENDPOINT_NAME,
                 temperature = 0.01,
                 language = language,
                 search_engine = search_engine,
                 model_type=model_type,
                 bedrock_api_url=api_url
                 )
print("finish init search_qa")

host: search-smartsearch-soghr3slgoubn2356ox22gitoy.us-west-1.es.amazonaws.com
region: us-west-1
finish init search_qa


In [18]:
# query='什么是Amazon SageMaker？'
# query='怎么部署呢？'
# query='what is aws s3?'
# query= '今天天气怎么样？'
# query= '美国有多少个州？'

# query='亚马逊云科技中国区域运营模式有什么特点？'
# query='亚马逊云科技有媒体行业解决方案吗？'
# query="游戏行业呢？"
# query='零售行业呢？'

query='What is Amazon Elastic Transcoder?'

top_k = 1
session_id='bedrock35'
table_name='LambdaStack-ChatSessionRecord189120C3-1HYZ55TUDXEUM'


result = search_qa.get_answer_from_conversational(query,
                            session_id,
                            table_name,
                            language,
                            prompt_template=prompt_template,
                            condense_question_prompt=condense_question_prompt,
                            top_k=top_k
                            )    
print('result:',result)
print('--------------------')
# print('answer:',result['answer'])
answer=result['answer'].split('\n\nhuman')[0].split('\n\n用户')[0].split('\n\nquestion')[0].split('\n\nQuestion')[0].strip()
print('answer:',answer)

history: []
prompt: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Question: Q: What is Amazon Elastic Transcoder?
Answer: Amazon Elastic Transcoder is a highly scalable, easy to use and cost effective way for developers and businesses to convert (or “transcode”) video and audio files from their source format into versions that will playback on devices like smartphones, tablets and PCs..

Question: What is Amazon Elastic Transcoder?
Answer:
_model_kwargs: {'modelId': 'anthropic.claude-v2', 'max_tokens': 500, 'temperature': 0.01}
******
bedrock prompt: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Question: Q: What is Amazon Elastic Transcoder?
Answer: Amazon Elastic Transcoder is a highly scalable, easy to use and cost effective way for developers and busin

In [26]:
source_documents = result['source_documents']
if search_engine == "opensearch":
    source_docs = [doc[0] for doc in source_documents]
    query_doc_scores = [doc[1] for doc in source_documents]
    sentences = [doc[2] for doc in source_documents]
elif search_engine == "kendra":
    source_docs = source_documents

In [27]:
answer = result['answer']        

if language == "english":
    answer = answer.split('Answer:')[-1]
print('answer:',answer)
print('query:',query)
#cal qa_relate_score
qa_relate_score = search_qa.get_qa_relation_score(query,answer)
print('1.qa_relate_score:',qa_relate_score)

answer:  AWS S3 is a fully managed object storage service provided by Amazon Web Services (AWS). It stands for Simple Storage Service and is designed to provide developers and IT teams with a scalable and high-performance way to store and retrieve data. AWS S3 is used by millions of customers worldwide and is the most widely used object storage service in the world.
query: what is aws s3?
1.qa_relate_score: 0.8680334150733228


In [28]:
answer_relate_scores = []
cal_answer = answer
if language == "chinese" and len(answer) > 150:
    cal_answer = answer[:150]
    
for source_doc in source_docs:
    cal_source_page_content = source_doc.page_content
    if language == "chinese" and len(cal_source_page_content) > 150:
        cal_source_page_content = cal_source_page_content[:150]
    answer_relate_score = search_qa.get_qa_relation_score(cal_answer,cal_source_page_content)
    answer_relate_scores.append(answer_relate_score)
print('2.answer_relate_scores:',answer_relate_scores)

2.answer_relate_scores: [0.6697910927220708, 0.5876096499497694, 0.6455764957531428]


In [29]:
#cal answer_relate_scores
find_answer_top_k = 3
answer_relate_docs = search_qa.get_retriever(find_answer_top_k).get_relevant_documents(answer)
print('answer_relate_docs:',answer_relate_docs)
if search_engine == "opensearch":
    answer_relate_docs = [doc[0] for doc in answer_relate_docs]

find_index = []
for answer_relate_doc in answer_relate_docs:
    try:
        relate_source = answer_relate_doc.metadata['source']
        relate_page_content = answer_relate_doc.page_content
    except KeyError:
        print("KeyError found")
        continue
    for i in range(len(source_docs)):
        source_page_content = source_docs[i].page_content
        if source_page_content == relate_page_content:
            find_index.append(i)
            break
print('find_index:',find_index)

if len(find_index) > 0:                
    list_score = 0
    find_topk = 1
    query_relate_docs_len = len(source_docs)
    answer_relate_docs_len = len(answer_relate_docs)
    for i in range(query_relate_docs_len):
        for j in range(len(find_index)):
            if i == find_index[j]:
                list_score += (query_relate_docs_len-i)*(answer_relate_docs_len-j)        
    print('qa_list_score:',list_score)

    total = query_relate_docs_len*answer_relate_docs_len
    if query_relate_docs_len > 1 and answer_relate_docs_len > 1:
        total += (query_relate_docs_len-1)*(answer_relate_docs_len-1)
    print('total score:',total)

    list_overlap_score = round(list_score/total,3)
else:
    list_overlap_score = 0.0
print('3.list_overlap_score:',list_overlap_score)
    


answer_relate_docs: [Document(page_content='Document Title: What is Amazon S3? - Amazon Simple Storage Service \nDocument Excerpt: \nIt is designed to make web-scale computing easier for developers. Amazon S3 has a simple web services interface that you can use to store and retrieve any amount of data, at any time, from anywhere on the web. It gives any developer access to the same highly scalable, reliable, fast, inexpensive data storage infrastructure that Amazon uses to run its own global network of web sites. The service aims to maximize benefits of scale and to pass those benefits on to developers. This guide explains the core concepts of Amazon S3, such as buckets, access points, and objects, and how to work with these resources using the Amazon S3 application programming interface (API). How do I? Information Relevant sections General product overview and pricing Amazon S3 Get a quick hands-on introduction to Amazon S3 Amazon Simple Storage Service Getting Started Guide Learn ab

In [30]:
source_list = []
for i in range(len(source_docs)):
    source = {}
    source["id"] = i
    try:
        source["source"] = os.path.split(source_docs[i].metadata['source'])[-1]
    except KeyError:
        print("KeyError found")                    
    source["paragraph"] =source_docs[i].page_content.replace("\n","")
    source["sentence"] = sentences[i] if search_engine == "opensearch" else source_docs[i].page_content.replace("\n","")
    source["score"] = query_doc_scores[i] if search_engine == "opensearch" else 1
    source_list.append(source)
print(source_list)

[{'id': 0, 'source': 'cloudtrail-logging.html', 'paragraph': "Document Title: Logging Amazon S3 API calls using AWS CloudTrail - Amazon Simple Storage Service Document Excerpt: AWSDocumentationAmazon Simple Storage Service (S3)Developer Guide Amazon S3 information in CloudTrailUsing CloudTrail logs with Amazon S3 server access logs and CloudWatch LogsExample: Amazon S3 log file entriesRelated resources Logging Amazon S3 API calls using AWS CloudTrail Amazon S3 is integrated with AWS CloudTrail, a service that provides a record of actions taken by a user, role, or an AWS service in Amazon S3. CloudTrail captures a subset of API calls for Amazon S3 as events, including calls from the Amazon S3 console and from code calls to the Amazon S3 APIs. API calls using AWS CloudTrail If you create a trail, you can enable continuous delivery of CloudTrail events to an Amazon S3 bucket, including events for Amazon S3. If you don't configure a trail, you can still view the most recent events in the C

In [31]:
source_list = []
for i in range(len(source_docs)):
    source = {}
    source["_id"] = i
    source["_score"] = query_doc_scores[i] if search_engine == "opensearch" else 1
    source["title"] = os.path.split(source_docs[i].metadata['source'])[-1]
    source["sentence"] = sentences[i] if search_engine == "opensearch" else source_docs[i].page_content.replace("\n","")
    source["paragraph"] =source_docs[i].page_content.replace("\n","")
    source["sentence_id"] = i
    if 'row' in source_docs[i].metadata.keys():
        source["paragraph_id"] = source_docs[i].metadata['row']
    elif 'page' in source_docs[i].metadata.keys():
        source["paragraph_id"] = source_docs[i].metadata['page']
    else:
        source["paragraph_id"] = i
    source_list.append(source)
print(source_list)

[{'_id': 0, '_score': 1, 'title': 'cloudtrail-logging.html', 'sentence': "Document Title: Logging Amazon S3 API calls using AWS CloudTrail - Amazon Simple Storage Service Document Excerpt: AWSDocumentationAmazon Simple Storage Service (S3)Developer Guide Amazon S3 information in CloudTrailUsing CloudTrail logs with Amazon S3 server access logs and CloudWatch LogsExample: Amazon S3 log file entriesRelated resources Logging Amazon S3 API calls using AWS CloudTrail Amazon S3 is integrated with AWS CloudTrail, a service that provides a record of actions taken by a user, role, or an AWS service in Amazon S3. CloudTrail captures a subset of API calls for Amazon S3 as events, including calls from the Amazon S3 console and from code calls to the Amazon S3 APIs. API calls using AWS CloudTrail If you create a trail, you can enable continuous delivery of CloudTrail events to an Amazon S3 bucket, including events for Amazon S3. If you don't configure a trail, you can still view the most recent eve